In [9]:
import ctypes
from ctypes import POINTER, c_double, c_int, c_void_p, c_int64

# Load the DLL
mlp_lib = ctypes.CDLL("../library/MyMLP_DLL_TEST.dll")

# Define types for the functions
create_mlp = mlp_lib.create_mlp
create_mlp.argtypes = [POINTER(c_int64), c_int64]
create_mlp.restype = ctypes.c_void_p

destroy_mlp = mlp_lib.destroy_mlp
destroy_mlp.argtypes = [ctypes.c_void_p]
destroy_mlp.restype = None

train = mlp_lib.train
train.argtypes = [c_void_p, POINTER(c_double), c_int, c_int, POINTER(c_double), c_int, c_double, c_int, ctypes.c_bool]

predict = mlp_lib.predict
predict.argtypes = [c_void_p, POINTER(c_double), c_int, POINTER(c_double), ctypes.c_bool]


In [10]:
X = [
    [0.0, 0.0],
    [0.0, 1.0],
    [1.0, 0.0]
]
Y = [
    [-1.0],
    [1.0],
    [-1.0]
]
# Create the MLP
layers = (c_int64 * 2)(2, 1)
mlp_instance = create_mlp(layers, 2)
# Conversion de X et Y en un format utilisable par ctypes
flat_X = (c_double * (len(X) * len(X[0])))(*sum(X, []))
flat_Y = (c_double * len(Y))(*(y[0] for y in Y))
train(mlp_instance, flat_X, len(X), len(X[0]), flat_Y, 1, 0.1, 1000000, True)

for i in range(len(X)):
    test_input = (c_double * len(X[0]))(*X[i])
    output = (c_double * 1)()
    predict(mlp_instance, test_input, len(X[0]), output, True)
    print("Predicted output for input", X[i], "is:", output[0])
destroy_mlp(mlp_instance)

Predicted output for input [0.0, 0.0] is: -0.9966798774210832
Predicted output for input [0.0, 1.0] is: 0.9972597942933524
Predicted output for input [1.0, 0.0] is: -0.9994545099334499


In [11]:
X = [
    [0.0, 0.0],
    [0.0, 1.0],
    [1.0, 0.0],
    [1.0, 1.0]
]

Y = [
    [-1.0],
    [1.0],
    [1.0],
    [-1.0]
]
layers = (c_int64 * 3)(2,3,1)
mlp_instance = create_mlp(layers, 3)

flat_X = (c_double * (len(X) * len(X[0])))(*sum(X, []))
flat_Y = (c_double * len(Y))(*(y[0] for y in Y))

train(mlp_instance, flat_X, len(X), len(X[0]), flat_Y, 1, 0.01, 1000000, True)
for i in range(len(X)):
    test_input = (c_double * len(X[0]))(*X[i])
    output = (c_double * 1)()
    predict(mlp_instance, test_input, len(X[0]), output, True)
    print("Predicted output for input", X[i], "is:", output[0])
destroy_mlp(mlp_instance)

Predicted output for input [0.0, 0.0] is: -0.99418732413861
Predicted output for input [0.0, 1.0] is: 0.9916717648824497
Predicted output for input [1.0, 0.0] is: 0.9940673471167212
Predicted output for input [1.0, 1.0] is: -0.9916722463964911


In [15]:
X = [
    [3.0],
    [7.0],
]

Y = [
    [8.0],
    [2.0],
]

# Create an MLP with a hidden layer
layers = (c_int64 * 3)(1, 3, 1)
mlp_instance = create_mlp(layers, 3)

# Flatten the inputs and outputs
flat_X = (c_double * (len(X) * len(X[0])))(*sum(X, []))
flat_Y = (c_double * len(Y))(*(y[0] for y in Y))

# Train the MLP
train(mlp_instance, flat_X, len(X), len(X[0]), flat_Y, len(Y[0]), 0.01, 1000000, False)

# Predict and print results
for i in range(len(X)):
    test_input = (c_double * len(X[0]))(*X[i])
    output = (c_double * 1)()
    predict(mlp_instance, test_input, len(X[0]), output, False)
    print("Predicted output for input", X[i], "is:", output[0])
destroy_mlp(mlp_instance)

Predicted output for input [3.0] is: 5.713054223348475
Predicted output for input [7.0] is: 5.713060263269588


In [6]:
X = [
    [3.0],
    [7.0],
    [9.0],
]

Y = [
    [8.0],
    [2.0],
    [6.0],
]
layers = (c_int64 * 3)(1, 3, 1)
mlp_instance = create_mlp(layers, 3)

# Flatten the inputs and outputs
flat_X = (c_double * (len(X) * len(X[0])))(*sum(X, []))
flat_Y = (c_double * len(Y))(*(y[0] for y in Y))

# Train the MLP
train(mlp_instance, flat_X, len(X), len(X[0]), flat_Y, len(Y[0]), 0.01, 1000000, False)

# Predict and print results
for i in range(len(X)):
    test_input = (c_double * len(X[0]))(*X[i])
    output = (c_double * 1)()
    predict(mlp_instance, test_input, len(X[0]), output, False)
    print("Predicted output for input", X[i], "is:", output[0])
destroy_mlp(mlp_instance)

Predicted output for input [3.0] is: 5.354773568485786
Predicted output for input [7.0] is: 5.3547820930243155
Predicted output for input [9.0] is: 5.354782093024838
